# LoRA applied to sentiment classification

In [1]:
import os
from collections import Counter
from tqdm import tqdm

import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_model, LoraConfig, TaskType, PromptTuningConfig, PromptTuningInit
from trl import SFTTrainer
import wandb

In [2]:
os.environ["WANDB_PROJECT"] = "lora_fingpt_sentiment_clf"
seed = 202408
set_seed(seed)

# huggingface endoints
device = "cuda"
model_name_or_path = "mistralai/Mistral-7B-v0.1"
tokenizer_name_or_path = "mistralai/Mistral-7B-v0.1"
dataset_name = "FinGPT/fingpt-sentiment-train"

# Dataset
text_column = "input"
label_column = "output"

# model hyperparams
max_length = 64
lr = 1e-4
num_epochs = 10
batch_size = 8

## Dataset prep

In [3]:
dataset = load_dataset(dataset_name)
# get a sample of dataset to accelerate training
dataset = dataset["train"].train_test_split(train_size=0.1)
classes = [row[label_column] for row in dataset["train"]]
print(set(classes))
print(dataset["train"][0])

{'strong positive', 'mildly negative', 'positive', 'neutral', 'moderately positive', 'moderately negative', 'mildly positive', 'strong negative', 'negative'}
{'input': 'Hedge Funds Have Never Been This Bullish On Chiasma Inc (CHMA)', 'output': 'positive', 'instruction': 'What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.'}


In [4]:
# instruction only states 3 possible labels {negative/neutral/positive}
# I assume available labels need to be enforced to be one of these 3

# define a dict to map labels
classes_dict = {}
for c in classes:
    if 'positive' in c:
        classes_dict[c] = 'positive'
    elif 'negative' in c:
        classes_dict[c] = 'negative'
    else:
        classes_dict[c] = 'neutral'

dataset = dataset.map(
    lambda x: {"text_label": [classes_dict[label] for label in x[label_column]]},
    batched=True,
    num_proc=1
)
print(set([row['text_label'] for row in dataset["train"]]))

Map:   0%|          | 0/7677 [00:00<?, ? examples/s]

Map:   0%|          | 0/69095 [00:00<?, ? examples/s]

{'neutral', 'negative', 'positive'}


In [5]:
# count labels proportions
Counter(dataset["train"]["text_label"])

Counter({'positive': 3066, 'neutral': 2914, 'negative': 1697})

In [6]:
# preprocess dataset
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes_dict.keys()])
print(f"target max length = ", target_max_length)

target max length =  4


In [7]:
instruction = dataset['train'][0]['instruction']
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x}\ninstruction : {instruction}\noutput : " for x in examples[text_column]]
    targets = [str(x) for x in examples["text_label"]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets, add_special_tokens=False) # dont add bos token cause this is concatenated with input
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.eos_token_id]
        #print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + \
            model_inputs["attention_mask"][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [8]:
# tokenize and preprocess dataset to be
# readable by the model as model inputs

preprocessed_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset"
)

# now get datasets
train_dataset = preprocessed_dataset["train"]
eval_dataset = preprocessed_dataset["train"] # there's no eval on the dataset

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
next(iter(train_dataloader))

Running tokenizer on dataset:   0%|          | 0/7677 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/69095 [00:00<?, ? examples/s]

{'input_ids': tensor([[    1,  2787,   714,   524,   581, 28720,  1440, 19912, 28713,   369,
          19357,   789, 28733,  8117, 21113,   662,  3428,  1618,   667,   622,
           2848,   871,  2545,   297,  1298,   789,   328, 28764,  1200,   297,
           7293,   842,    13,  4138,  3112,   714,  1824,   349,   272, 21790,
            302,   456,  9394,   299, 28804,  5919,  4987,   396,  4372,   477,
            371, 23238, 28748, 23806,  1650, 28748,  1065,  2468,  2051,    13,
           4050,   714, 28705, 14214],
         [    1,  2787,   714,  3651,   529,   266,   464, 28713,  4098,   420,
           8423, 28740, 28790,   349,  9206,   356,   272, 18064,  5569, 28824,
            451, 26534,   382,  1190,   655, 28710,  1200, 11083,  6275,  3231,
            842,    13,  4138,  3112,   714,  1824,   349,   272, 21790,   302,
            456,  9394,   299, 28804,  5919,  4987,   396,  4372,   477,   371,
          23238, 28748, 23806,  1650, 28748,  1065,  2468,  2051,   

In [9]:
#def test_preprocess_function(examples):
#    batch_size = len(examples[text_column])
#    inputs = [f"{text_column}: {x}\noutput" for x in examples[text_column]]
#    model_inputs = tokenizer(inputs)
#    for i in range(batch_size):
#        sample_inputs_ids = model_inputs["input_ids"][i]
#        model_inputs["input_ids"][i] = [tokenizer.pad_token_id]*(
#            max_length - len(sample_inputs_ids)
#        ) + sample_inputs_ids
#        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_inputs_ids)) + \
#            model_inputs["attention_mask"][i]
#        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
#        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
#    
#    return model_inputs

## Create PEFT model

In [10]:
# this instruction is from the dataset
#prompt_tunning_init_text = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}. \n"
target_modules = [
    "gate_proj",
    "q_proj",
    "lm_head",
    "o_proj",
    "k_proj",
    "embed_tokens",
    "down_proj",
    "up_proj",
    "v_proj"
]

# now create peft config
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=target_modules,
    task_type=TaskType.CAUSAL_LM
)

In [11]:
# create model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16
)
model.resize_token_embeddings(len(tokenizer))
model = get_peft_model(model, peft_config)
# see trainable params
model.print_trainable_parameters()
# cast non-trainable parameters to fp16
for p in model.parameters():
    if not p.requires_grad:
        p.data = p.to(torch.float16)
# enable model gradient checkpointing
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
# pass model to GPU
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 21,549,056 || all params: 7,263,281,152 || trainable%: 0.2967


In [12]:
# define optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.1)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader)*num_epochs)
)

## Train and eval loop

In [15]:
wandb.init(project='lora_fingpt_sentiment_clf', name='lora')

# training and evaluation
for epoch in range(num_epochs):
    # state model on train stage
    model.train()
    # this will count loss over each epoch
    total_loss = 0
    # start iterating over each batch on train dataloader
    for step, batch in enumerate(tqdm(train_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.autocast(dtype=torch.float16, device_type="cuda"):
            # map batch values to model and get outputs on forward pass
            outputs = model(**batch)
        # get loss from output
        loss = outputs.loss
        # start accumulating losses over each epoch
        total_loss += loss.detach().float()
        # do backward step
        loss.backward()
        # optimize on backward step
        optimizer.step()
        # define if learning rate must be redefine on this step
        lr_scheduler.step()
        # reset gradients for next step
        optimizer.zero_grad()

    # state model on evaluation stage
    model.eval()
    # this will count loss over each epoch for evaluation set
    eval_loss = 0
    eval_preds = []
    # start iterating over each batch on evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # send batch values to GPU
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            # use no gradients for this cause this is eval only
            # map batch values to model and get outputs
            outputs = model(**batch)
        # get loss from evaluation outputs
        loss = outputs.loss
        # start accumulating losses over each epoch
        eval_loss += loss.detach().float()
        # storage predictions
        eval_preds.extend(
            tokenizer.batch_decode(
                torch.argmax(outputs.logits, -1).detach().cpu().numpy(),
                skip_special_tokens=True
            )
        )

    # get average of eval losses
    eval_epoch_loss = eval_loss / len(eval_dataloader)
    # get perpelexity of eval losses
    # perplexity: measure of how well a model is accomplishing its task.
    # read further info here: https://developers.google.com/machine-learning/glossary#perplexity
    eval_ppl = torch.exp(eval_epoch_loss)
    # get average of train losses
    train_epoch_loss = total_loss / len(train_dataloader)
    # get perpelexity of train losses
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

    # log metrics on wandb
    wandb.log(
        {
            "train": {"perplexity": train_ppl, "loss": train_epoch_loss, "epoch": epoch},
            "val": {"perplexity": eval_ppl, "loss": eval_epoch_loss, "epoch": epoch} 
        }
    )

wandb: Currently logged in as: jdgallegoq (jdgallegoq-wandb). Use `wandb login --relogin` to force relogin


100%|██████████| 960/960 [02:27<00:00,  6.53it/s]


epoch=0: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


100%|██████████| 960/960 [02:26<00:00,  6.54it/s]


epoch=1: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


100%|██████████| 960/960 [02:27<00:00,  6.53it/s]


epoch=2: train_ppl=tensor(nan, device='cuda:0') train_epoch_loss=tensor(nan, device='cuda:0') eval_ppl=tensor(nan, device='cuda:0') eval_epoch_loss=tensor(nan, device='cuda:0')


  1%|          | 11/960 [00:05<07:27,  2.12it/s]


KeyboardInterrupt: 

## Evaluation after finetuning

In [19]:
model.eval()
i = 100
inputs = tokenizer(
    f"{text_column}: {dataset['train'][i][text_column]}\ninstruction : {instruction}\noutput : ",
    return_tensors="pt"
)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=target_max_length+1,
        eos_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=False)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> input: The parties have also agreed on options that allow UH to extensively implement later on also functionalities of human resources management .
instruction : What is the sentiment of this tweet? Please choose an answer from {negative/neutral/positive}.
output :  positive</s>
